# Compare DMS to natural sequence evolution

In [1]:
# this cell is tagged parameters for papermill parameterization
dms_summary_csv = None
pango_consensus_seqs_json = None
fitness_csv = None
starting_clade = None

In [2]:
import os
os.chdir("../")

dms_summary_csv = "summary.csv"
pango_consensus_seqs_json = "pango-consensus-sequences_summary.json"
fitness_csv = "aa_fitness.csv"
starting_clade = "XBB"
dms_clade = "XBB.1.5"

In [3]:
import collections
import json
import re

import altair as alt

import pandas as pd

## Read Pango clades and mutations

In [4]:
with open(pango_consensus_seqs_json) as f:
    pango_clades = json.load(f)

def n_children(c):
    """Get number of children clades of a Pango clade."""
    direct_children = pango_clades[c]["children"]
    return len(direct_children) + sum([n_children(c_child) for c_child in direct_children])

def build_records(c, recs):
    """Build records of Pango clade information."""
    recs["clade"].append(c)
    recs["n_children"].append(n_children(c))
    recs["date"].append(pango_clades[c]["designationDate"])
    recs["mutations_from_ref"].append(
        [
            mut.split(":")[1]
            for field in ["aaSubstitutions", "aaDeletions"]
            for mut in pango_clades[c][field]
            if mut.startswith("S:")
        ]
    )
    for c_child in pango_clades[c]["children"]:
        build_records(c_child, recs)
        
records = collections.defaultdict(list)
build_records(starting_clade, records)

pango_df = pd.DataFrame(records)
starting_clade_mutations_from_ref = pango_df.set_index("clade").at[
    starting_clade, "mutations_from_ref"
]
dms_clade_mutations_from_ref = pango_df.set_index("clade").at[
    dms_clade, "mutations_from_ref"
]

def mutations_from(muts, from_muts):
    """Get mutations from another sequence."""
    new_muts = set(muts).symmetric_difference(from_muts)
    assert all(re.fullmatch("[A-Z\-]\d+[A-Z\-]", m) for m in new_muts)
    new_muts_d = collections.defaultdict(list)
    for m in new_muts:
        new_muts_d[int(m[1: -1])].append(m)
    new_muts_list = []
    for _, ms in sorted(new_muts_d.items()):
        if len(ms) == 1:
            m = ms[0]
            if m in muts:
                new_muts_list.append(m)
            else:
                assert m in from_muts
                new_muts_list.append(m[-1] + m[1: -1] + m[0])
        else:
            m, from_m = ms
            if m not in muts:
                from_m, m = m, from_m
            assert m in muts and from_m in from_muts
            new_muts_list.append(from_m[-1] + m[1: ])
    return new_muts_list

pango_df = (
    pango_df
    .assign(
        mutations_from_start_clade=lambda x: x["mutations_from_ref"].apply(
            mutations_from, args=(starting_clade_mutations_from_ref,),
        ).map(lambda ml: "; ".join(ml)),
        mutations_from_dms_clade=lambda x: x["mutations_from_ref"].apply(
            mutations_from, args=(dms_clade_mutations_from_ref,),
        ),
        date=lambda x: pd.to_datetime(x["date"]),
    )
    .drop(columns="mutations_from_ref")
    .rename(columns={"mutations_from_start_clade": f"mutations_from_{starting_clade}"})
    .sort_values("date")
    .reset_index(drop=True)
)

pango_df

,clade,n_children,date,mutations_from_XBB,mutations_from_dms_clade
0,XBB,606,2022-09-17,,"[V252G, P486S]"
1,XBB.1,528,2022-10-03,G252V,[P486S]
2,XBB.1.1,0,2022-10-15,G252V,[P486S]
3,XBB.2,60,2022-10-15,D253G,"[V252G, D253G, P486S]"
4,XBB.3,5,2022-10-15,,"[V252G, P486S]"
...,...,...,...,...,...
602,HK.11,0,2023-09-04,Q52H; G184S; G252V; F456L; S486P,"[Q52H, G184S, F456L]"
603,XBB.1.16.24,0,2023-09-04,E180V; K182E; G252V; K478R; S486P; P521T; E554K,"[E180V, K182E, K478R, P521T, E554K]"
604,JG.2,0,2023-09-04,Q52H; G75V; G252V; F456L; S486P,"[Q52H, G75V, F456L]"
605,FY.4.1.2,0,2023-09-06,Y200C; G252V; Y451H; S486P; S494P,"[Y200C, Y451H, S494P]"


## Assign DMS phenotypes